# **Homework orchestration**

## Question 1. Sélectionnez l'outil

In [ ]:
import prefect
import pandas as pd
import mlflow
import mlflow.sklearn
import os
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression

## Question 2. Version

In [4]:
prefect.__version__

'3.4.4'

## Question 3. Créer un pipeline

In [7]:


def load_data():
    df = pd.read_parquet('yellow_tripdata_2023-03.parquet')
    print(f"Nombre d'enregistrements chargés : {df.shape[0]}")
    return df

df = load_data()

Nombre d'enregistrements chargés : 3403766


## Question 4. Préparation des données

In [8]:
def read_dataframe(filename):
    df = pd.read_parquet(filename)

    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df.duration = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df

df = read_dataframe('yellow_tripdata_2023-03.parquet')

In [9]:
df.shape

(3316216, 20)

In [ ]:


def train_model(df):
    train_dicts = df[['PULocationID', 'DOLocationID']].to_dict(orient='records')

    dv = DictVectorizer()
    X_train = dv.fit_transform(train_dicts)
    y_train = df['duration'].values

    model = LinearRegression()
    model.fit(X_train, y_train)

    print(f"Intercept du modèle : {round(model.intercept_, 2)}")
    return dv, model

dv, model = train_model(df)


Intercept du modèle : 24.78


In [ ]:
with mlflow.start_run():
    mlflow.sklearn.log_model(model, artifact_path="model")
    model_uri = mlflow.get_artifact_uri("model")

# Pour lire la taille du fichier .pkl
model_path = os.path.join(model_uri.replace("file://", ""), "model.pkl")
print(f"Taille du modèle : {os.path.getsize(model_path)} bytes")

ModuleNotFoundError: No module named 'distutils._modified'

9,534 bytes